In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from src import load_data, transform_data

# The datasets

Loading the summary Excel file

In [4]:
# Already existed
summary_data = load_data.get_summary_data("data/raw/summary.xlsx")
summary_data.head()

,Duration,Sends,Opens,Clicks,Open Rate,Click Rate,Unsubscribes,Word Count,Link Count,Month,Day,Year,Week,Weekday
Date/Time,,,,,,,,,,,,,,
2020-07-20 06:47:17.764,0:00:25.135,87,102,6,1.1724,0.0588,0,450,21,7,20,2020,30,0
2020-07-21 06:33:50.176,0:00:33.135,74,142,22,1.9189,0.1549,0,478,21,7,21,2020,30,1
2020-07-22 06:28:15.271,0:00:33.872,78,123,41,1.5769,0.3333,0,496,21,7,22,2020,30,2
2020-07-23 06:38:34.308,0:00:33.712,82,135,7,1.6463,0.0519,0,496,24,7,23,2020,30,3
2020-07-24 08:19:54.483,0:00:35.835,83,270,15,3.2530,0.0556,0,515,18,7,24,2020,30,4


Loading the consolidated article click data

In [2]:
# Derived from the article click Excel files from Google Drive
click_data = pd.read_csv("data/processed/Consolidated Article Click Data.csv")
click_data.Date = click_data.Date.astype(dtype=np.datetime64)
click_data.head()

,Date,Tag,Link,Clicks,Source,Month,Day,Year,Week,Weekday
0,2021-02-15,0,https://thecolumn.co/daily/02152021,36.0,2152021,2,15,2021,7,0
1,2021-02-15,1,https://thecolumn.co/,0.0,2152021,2,15,2021,7,0
2,2021-02-15,2,https://thecolumn.co/,35.0,2152021,2,15,2021,7,0
3,2021-02-15,3,https://commons.wikimedia.org/wiki/File:Comput...,4.0,2152021,2,15,2021,7,0
4,2021-02-15,4,http://NationalArchives.gov.uk/doc/open-govern...,8.0,2152021,2,15,2021,7,0


If the click data is not available, then it can be recreated by running the cell below

In [10]:
click_data = load_data.get_article_click_data("data/raw/article clicks")
click_data.to_csv("data/processed/All Click Data.csv")

Downloading the article HTML files, if they are not already downloaded

In [ ]:
# Web scraped from The Column website
article_section_list = dict()
link_info = dict()
date_url_endpoints = [transform_data.form_url_date(idx) for idx in summary_data.index]
for url_endpoint_date in date_url_endpoints:
    try:
        load_data.write_article_to_file(url_endpoint_date)
    except Exception as ex:
        print(url_endpoint_date, "failed to be loaded")
        print(ex)

Getting article text and links by section in the articles

These are the sections that were processed:
- Story/Article 1
- Story/Article 2
- Story/Article 3
- Other Headlines (first appears on 12/28/2020)
- Molecule of the Day (MOTD) (first appears on 12/28/2020)

Parsing the HTML files for section content and links

In [5]:
article_section_list = dict()
link_info = dict()
date_url_endpoints = [transform_data.form_url_date(idx) for idx in summary_data.index]
for url_endpoint_date in date_url_endpoints:
    try:
        article_section_list[url_endpoint_date], link_info[url_endpoint_date] = load_data.get_article_content_by_section(url_endpoint_date)
    except Exception as ex:
        print(url_endpoint_date, "failed to be loaded")
        print(ex)

/Users/benperez/Documents/The Column Data Hackathon 2021-10-23/DashWebApp/src/load_data.py:53: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 53 of the file /Users/benperez/Documents/The Column Data Hackathon 2021-10-23/DashWebApp/src/load_data.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  url_soup = BeautifulSoup(html_file)


Transforming the results of the previous cell into a DataFrame

In [47]:
article_section_data = transform_data.get_article_section_data(article_section_list, link_info)
article_section_data.Date = article_section_data.Date.apply(lambda dt : datetime.strptime(dt, "%m%d%Y"))
article_section_data["Weekday"] = article_section_data.Date.dt.weekday
article_section_data.head()

,Date,ArticleNumber,SectionText,LinkCount,SectionArticleLength,Weekday
0,2020-07-20,0,The European Commission has fined US-based Cel...,6,849,0
1,2020-07-20,1,"Poland's state-controlled largest refiner, PKN...",5,832,0
2,2020-07-20,2,Perfect Day has now raised their total investm...,4,852,0
3,2020-07-21,0,"Standard Oil successor, Chevron, announced yes...",8,846,1
4,2020-07-21,1,"Way back in 1964, Irving-based Celanese formed...",3,910,1


In [8]:
link_info_data = transform_data.get_link_info_data(link_info)
link_info_data.Date = link_info_data.Date.apply(lambda dt : datetime.strptime(dt, "%m%d%Y"))
link_info_data.head()

,Date,ArticleNumber,LinkText,LinkHref
0,2020-07-20,0,Celanese,https://en.wikipedia.org/wiki/Celanese
1,2020-07-20,0,Clariant,https://en.wikipedia.org/wiki/Clariant
2,2020-07-20,0,Orbia,https://en.wikipedia.org/wiki/Orbia
3,2020-07-20,0,Westlake Chemical,https://en.wikipedia.org/wiki/Westlake_Chemical
4,2020-07-20,0,by steam cracking natural gas,https://en.wikipedia.org/wiki/Ethylene#Industr...


In [11]:
link_click_data = transform_data.get_click_link_data(click_data, link_info_data)
link_click_data.head()
# If link_click_data not downloaded
#link_click_data.to_csv("data/processed/Link Click Data.csv")

,Date,ArticleNumber,LinkText,Tag,Link,Clicks,Month,Day,Year,Week,Weekday
0,2020-07-20,0,Celanese,3,https://en.wikipedia.org/wiki/Celanese,0.0,7,20,2020,30,0
1,2020-07-20,0,Clariant,4,https://en.wikipedia.org/wiki/Clariant,0.0,7,20,2020,30,0
2,2020-07-20,0,Orbia,5,https://en.wikipedia.org/wiki/Orbia,0.0,7,20,2020,30,0
3,2020-07-20,0,Westlake Chemical,6,https://en.wikipedia.org/wiki/Westlake_Chemical,1.0,7,20,2020,30,0
4,2020-07-20,0,by steam cracking natural gas,7,https://en.wikipedia.org/wiki/Ethylene#Industr...,0.0,7,20,2020,30,0


In [34]:
# assign a negative 1 to all newsletters that do not have "other headlines" and "molecule of the day" section
click_sum = link_click_data.groupby(["Date","ArticleNumber"]).sum()["Clicks"]
article_click_sum = click_sum.reset_index()
article_click_sum

,Date,ArticleNumber,Clicks
0,2020-07-20,0,1.0
1,2020-07-20,1,0.0
2,2020-07-20,2,2.0
3,2020-07-21,0,10.0
4,2020-07-21,1,4.0
...,...,...,...
929,2021-10-11,0,171.0
930,2021-10-11,1,51.0
931,2021-10-11,2,28.0
932,2021-10-11,3,57.0


In [48]:
article_section_data = article_section_data.merge(article_click_sum, on=["Date","ArticleNumber"], how="inner")
# If the article section data is not present in the data folder, then uncomment the line below
article_section_data.to_csv("data/processed/Article Section Data.csv")

In [49]:
article_section_data

,Date,ArticleNumber,SectionText,LinkCount,SectionArticleLength,Weekday,Clicks
0,2020-07-20,0,The European Commission has fined US-based Cel...,6,849,0,1.0
1,2020-07-20,1,"Poland's state-controlled largest refiner, PKN...",5,832,0,0.0
2,2020-07-20,2,Perfect Day has now raised their total investm...,4,852,0,2.0
3,2020-07-21,0,"Standard Oil successor, Chevron, announced yes...",8,846,1,10.0
4,2020-07-21,1,"Way back in 1964, Irving-based Celanese formed...",3,910,1,4.0
...,...,...,...,...,...,...,...
929,2021-10-11,0,"Colorado-based renewable chemicals company, Ge...",9,1063,0,171.0
930,2021-10-11,1,"Industrial gases company, Air Products, has an...",5,1170,0,51.0
931,2021-10-11,2,"Japanese-based chemical company, Toray, has an...",7,922,0,28.0
932,2021-10-11,3,Neste will now sell its renewable diesel at 7 ...,5,356,0,57.0


In [22]:
click_sum

ArticleNumber,0,1,2,3,4,Weekday
Date,,,,,,
2020-07-20,1.0,0.0,2.0,-1.0,-1.0,0
2020-07-21,10.0,4.0,5.0,-1.0,-1.0,1
2020-07-22,1.0,2.0,33.0,-1.0,-1.0,2
2020-07-23,3.0,1.0,1.0,-1.0,-1.0,3
2020-07-24,1.0,10.0,3.0,-1.0,-1.0,4
...,...,...,...,...,...,...
2021-10-01,100.0,38.0,31.0,34.0,31.0,4
2021-10-04,105.0,64.0,13.0,47.0,33.0,0
2021-10-06,103.0,94.0,23.0,54.0,38.0,2


# Transferring data from flat files to SQLite

In [12]:
section_data_path = "data/processed/Article Section Data.csv"
summary_data_path = "data/raw/summary.xlsx"
article_section_data = pd.read_csv(section_data_path, parse_dates=True, index_col="Date")
article_section_data.drop(labels="Unnamed: 0", axis=1, inplace=True)
summary_data = load_data.get_summary_data(summary_data_path)

In [19]:
summary_data["Date"] = summary_data.index.date
summary_data.head()

,Duration,Sends,Opens,Clicks,Open Rate,Click Rate,Unsubscribes,Word Count,Link Count,Month,Day,Year,Week,Weekday,Date
Date/Time,,,,,,,,,,,,,,,
2020-07-20 06:47:17.764,0:00:25.135,87,102,6,1.1724,0.0588,0,450,21,7,20,2020,30,0,2020-07-20
2020-07-21 06:33:50.176,0:00:33.135,74,142,22,1.9189,0.1549,0,478,21,7,21,2020,30,1,2020-07-21
2020-07-22 06:28:15.271,0:00:33.872,78,123,41,1.5769,0.3333,0,496,21,7,22,2020,30,2,2020-07-22
2020-07-23 06:38:34.308,0:00:33.712,82,135,7,1.6463,0.0519,0,496,24,7,23,2020,30,3,2020-07-23
2020-07-24 08:19:54.483,0:00:35.835,83,270,15,3.2530,0.0556,0,515,18,7,24,2020,30,4,2020-07-24


In [9]:
article_section_data.head()

,ArticleNumber,SectionText,LinkCount,SectionArticleLength,Weekday,Clicks
Date,,,,,,
2020-07-20,0,The European Commission has fined US-based Cel...,6,849,0,1.0
2020-07-20,1,"Poland's state-controlled largest refiner, PKN...",5,832,0,0.0
2020-07-20,2,Perfect Day has now raised their total investm...,4,852,0,2.0
2020-07-21,0,"Standard Oil successor, Chevron, announced yes...",8,846,1,10.0
2020-07-21,1,"Way back in 1964, Irving-based Celanese formed...",3,910,1,4.0


In [20]:
sqlite_file = "data.db"
summary_table_name = "Summary"
article_section_table = "ArticleSection"
conn = sqlite3.connect(sqlite_file)
summary_data.to_sql(summary_table_name, conn, if_exists="replace")
article_section_data.to_sql(article_section_table, conn, if_exists="replace")

/Users/benperez/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [3]:
sqlite_file = "data.db"
conn = sqlite3.connect(sqlite_file)
summary_table_name = "Summary"
article_section_table = "ArticleSection"
query = '''
        SELECT *
        FROM Summary
        '''
query_result_summary = pd.read_sql(query, conn, index_col="Date")
query_result_summary

,Date/Time,Duration,Sends,Opens,Clicks,Open Rate,Click Rate,Unsubscribes,Word Count,Link Count,Month,Day,Year,Week,Weekday
Date,,,,,,,,,,,,,,,
2020-07-20,2020-07-20 06:47:17.764000,0:00:25.135,87,102,6,1.1724,0.0588,0,450,21,7,20,2020,30,0
2020-07-21,2020-07-21 06:33:50.176000,0:00:33.135,74,142,22,1.9189,0.1549,0,478,21,7,21,2020,30,1
2020-07-22,2020-07-22 06:28:15.271000,0:00:33.872,78,123,41,1.5769,0.3333,0,496,21,7,22,2020,30,2
2020-07-23,2020-07-23 06:38:34.308000,0:00:33.712,82,135,7,1.6463,0.0519,0,496,24,7,23,2020,30,3
2020-07-24,2020-07-24 08:19:54.483000,0:00:35.835,83,270,15,3.2530,0.0556,0,515,18,7,24,2020,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01,2021-10-01 09:34:55.009000,0:21:23.488,3648,1588,245,0.4353,0.1543,1,536,45,10,1,2021,39,4
2021-10-04,2021-10-04 07:47:17.811000,0:15:51.485,3674,1978,307,0.5384,0.1552,2,449,42,10,4,2021,40,0
2021-10-06,2021-10-06 08:41:22.988000,0:18:06.223,3718,1748,333,0.4701,0.1905,4,499,49,10,6,2021,40,2


In [35]:
def format_select_query(query_args):
      query = '''
            SELECT *
            FROM {0}
            WHERE Date >= '{1}' AND
                  Date <= '{2}' 
            '''
      return query.format(*query_args)

start_date = '2021-01-01'
end_date = '2021-09-01'
table_name = "Summary"
query_result_article = pd.read_sql(format_select_query([table_name, start_date, end_date]), conn, index_col="Date")
query_result_article

,Date/Time,Duration,Sends,Opens,Clicks,Open Rate,Click Rate,Unsubscribes,Word Count,Link Count,Month,Day,Year,Week,Weekday
Date,,,,,,,,,,,,,,,
2021-01-01,2021-01-01 08:10:30.409000,0:08:33.112,1379,618,121,0.4482,0.1958,1,476,45,1,1,2021,53,4
2021-01-04,2021-01-04 08:14:41.016000,0:13:38.021,1396,899,204,0.6440,0.2269,1,456,44,1,4,2021,1,0
2021-01-06,2021-01-06 07:41:32.220000,0:15:49.795,1400,731,139,0.5221,0.1902,1,439,43,1,6,2021,1,2
2021-01-08,2021-01-08 08:46:45.244000,0:09:56.552,1400,793,106,0.5664,0.1337,0,430,40,1,8,2021,1,4
2021-01-11,2021-01-11 08:00:58.638000,0:10:30.952,1404,819,152,0.5833,0.1856,3,494,51,1,11,2021,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-23,2021-08-23 07:16:16.398000,0:12:58.426,2912,1559,276,0.5354,0.1770,2,585,62,8,23,2021,34,0
2021-08-25,2021-08-25 07:17:33.756000,0:12:25.358,2933,1534,257,0.5230,0.1675,2,514,54,8,25,2021,34,2
2021-08-27,2021-08-27 09:27:37.016000,0:14:19.472,2948,1290,220,0.4376,0.1705,0,548,52,8,27,2021,34,4


In [5]:
def format_agg_query(query_args):
      query = '''
            SELECT {0}, {1}({2}) As {2}
            FROM {3}
            WHERE Date >= '{4}' AND
                  Date <= '{5}'
            GROUP BY {0}
            '''
      return query.format(*query_args)

group_column = "Weekday, ArticleNumber"
agg_column = "Clicks"
agg_func_name = "AVG"
start_date = '2021-08-01'
end_date = '2021-09-01'
table_name = "ArticleSection"
query_result_article = pd.read_sql(format_agg_query([group_column, agg_func_name, agg_column, table_name, start_date, end_date]), conn)
query_result_article

,Weekday,ArticleNumber,Clicks
0,0,0,89.80
1,0,1,69.60
2,0,2,40.20
3,0,3,37.00
4,0,4,39.00
5,2,0,127.75
6,2,1,36.50
7,2,2,22.25
8,2,3,26.00
9,2,4,31.00


In [11]:
weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
weekday_dict = {}
for i, j in enumerate(weekdays):
    weekday_dict[i] = j
query_result_article.Weekday.map(weekday_dict)
query_result_article.Weekday = query_result_article.Weekday.map(weekday_dict)
query_result_article

,Weekday,ArticleNumber,Clicks
0,Mon,0,89.80
1,Mon,1,69.60
2,Mon,2,40.20
3,Mon,3,37.00
4,Mon,4,39.00
5,Wed,0,127.75
6,Wed,1,36.50
7,Wed,2,22.25
8,Wed,3,26.00
9,Wed,4,31.00


## Testing Data Uploads

In [4]:
upload_data = pd.read_excel("data/raw/uploads/01152021.xlsx")
upload_data

,Tag,Link,Clicks
0,0,https://thecolumn.co/daily/01152021,26
1,1,https://thecolumn.co/,0
2,2,https://thecolumn.co/,27
3,3,https://commons.wikimedia.org/wiki/File:Divers...,2
4,4,https://creativecommons.org/licenses/by/4.0,2
5,5,https://en.wikipedia.org/wiki/Celanese,3
6,6,https://en.wikipedia.org/wiki/Liquid-crystal_p...,3
7,7,https://www.polyplastics-global.com/en/aboutus...,3
8,8,https://thecolumn.co/daily/07212020,26
9,9,0,0
